In [1]:
# Imports und Pfade

import os
import json

from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage, SystemMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

# Pfad zum Mistral-Modell
# PROJECT_ROOT = "/sc-projects/sc-proj-dhzc-psycho/LLM-PSY"
mistral_models_path = "/sc-resources/llms/mistralai/Mistral-7B-Instruct-v0.3"


In [2]:
# Modell und Tokenizer laden
tokenizer = MistralTokenizer.from_file(
    os.path.join(mistral_models_path, "tokenizer.model.v3")
)

model = Transformer.from_folder(mistral_models_path)

In [3]:
# Prompt (System + User) definieren

system_content = (
    "Du erhältst ein Teil des Interviewtranskripts, und als Kontext die Zusammenfassung des Interviews und die vorherige Ausgabe. "
    "Identifiziere psychiatrisch relevante Symptome für Major Depression Episode und Bipolare Störungen (DSM-5), und zitiere die jeweiligen Abschnitte (Wortlaute), die jedes Symptom belegen. "
    "Gib nur gültiges JSON als Liste von Objekten zurück: "
    "[{'symptom': '<label>', 'section': '<exact transcript quote>'}, {'symptom': '<label>', 'section': '<exact transcript quote>'}, ...]. "
    "Verwende ausschließlich folgende Symptome: "
    "Schwere depressive Episode: Depressive Stimmung, Interessen- oder Freudeverlust, Verminderter oder gesteigerter Appetit, Gewichtsverlust oder -zunahme, "
    "Insomnie oder Hypersomnie, Psychomotorische Unruhe oder Verlangsamung, Müdigkeit oder Energieverlust, Schuld- oder Wertlosigkeitsgefühle, "
    "Konzentrations- oder Entscheidungsschwierigkeiten, Suizidalität. "
    "Manische und hypomanische Episode: Gereizte oder gehobene oder expansive Stimmung, Gesteigertes Selbstwertgefühl oder Größenideen, Vermindertes Schlafbedürfnis, "
    "Gesteigertes Sprechbedürfnis oder Rededruck, Ideenflucht oder Gedankenrasen, Ablenkbarkeit, "
    "Gesteigerte zielgerichtete Aktivität oder psychomotorische Unruhe, Risikoverhalten. "
    "Gebe die Abschnitte wörtlich aus dem Transkript wieder (keine Zusammenfassung). Gib nur die minimal nötigen Wörter zur Belegung des Symptoms (keine ganzen Sätze, wenn nicht nötig). "
    "Wenn ein Abschnitt mehrere Symptome enthält, liste sie zusammen auf: [{'symptom': '..., ...', 'section': '...'}, ...]. "
    "Ignoriere negierte Symptome, sofern sie nicht an anderer Stelle widerlegt werden, und konzentriere dich auf Informationen des Patienten. "
    "Wenn keine psychiatrisch relevanten Symptome vorliegen, antworte mit: [{'symptom': 'none', 'section': 'none'}]. "
    "Dein Output darf maximal 250 Wörter enthalten."
)

# Dummy-Beispiel für einen Transkript-Ausschnitt
transcript_excerpt = (
    "Patient: Ich schlafe seit Wochen kaum mehr und bin ständig voller Energie. "
    "Ich rede sehr schnell, meine Freunde sagen, sie können mir kaum folgen. "
    "Gleichzeitig habe ich manchmal Tage, an denen ich mich völlig wertlos fühle."
)

user_content = (
    "Hier sind der Transkriptteil und Kontextinformationen. "
    "Extrahiere die Symptome und unterstützenden Abschnitte wie angegeben.\n\n"
    f"TRANSKRIPT:\n{transcript_excerpt}"
)

messages = [
    SystemMessage(content=system_content),
    UserMessage(content=user_content),
]

completion_request = ChatCompletionRequest(messages=messages)

In [4]:
# Anfrage an Mistral und Antwort decodieren

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate(
    [tokens],
    model,
    max_tokens=256,      # wie im CLI-Beispiel
    temperature=0.0,     # deterministisch für Evaluation
    eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id,
)

raw_output = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])
print("Roh-Output:")
print(raw_output)


Roh-Output:
[
  {'symptom': 'Insomnie oder Hypersomnie, Psychomotorische Unruhe oder Verlangsamung, Gesteigertes Sprechbedürfnis oder Rededruck', 'section': 'Ich schlafe seit Wochen kaum mehr und bin ständig voller Energie. Ich rede sehr schnell, meine Freunde sagen, sie können mir kaum folgen.'},
  {'symptom': 'Schuld- oder Wertlosigkeitsgefühle', 'section': 'Gleichzeitig habe ich manchmal Tage, an denen ich mich völlig wertlos fühle.'}
]


In [5]:
# JSON-Parsing

def parse_json_from_response(response_text: str):
    """
    Versucht, die Modellantwort als JSON zu interpretieren.
    Erwartet eine Liste von Objekten [{...}, {...}, ...].
    """
    response_text = response_text.strip()
    
    # Optional: Falls das Modell Text vor/nach dem JSON ausgibt, könnte man
    # hier noch heuristisch den ersten '[' und letzten ']' suchen.
    try:
        data = json.loads(response_text)
    except json.JSONDecodeError:
        # Fallback: Versuche, nur den JSON-Teil zwischen [ und ] zu nehmen
        start = response_text.find("[")
        end = response_text.rfind("]")
        if start != -1 and end != -1 and end > start:
            try:
                data = json.loads(response_text[start:end+1])
            except json.JSONDecodeError:
                data = None
        else:
            data = None
    return data

parsed = parse_json_from_response(raw_output)
print("\nGeparstes JSON:")
print(parsed)



Geparstes JSON:
None
